In [103]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from IPython.display import clear_output

In [118]:
# import precipitation data for August 2015

precip_folder_path = './csv/daily'
csv_precip_files = [file for file in os.listdir(precip_folder_path) if file.endswith('.csv')]
precip_dataframes = {}

for csv_file in csv_precip_files:

    day = int(csv_file.split('-')[2].split('.')[0])
    
    df = pd.read_csv(os.path.join(precip_folder_path, csv_file))
    
    precip_dataframes[f'{day}'] = df

In [119]:
# Round latitude/longitude values to 2 decimal places

for df in precip_dataframes.values():
    df['lon'] = df['lon'].apply(lambda x: round(x,2))
    df['lat'] = df['lat'].apply(lambda x: round(x,2))

precip_dataframes['1'].size

37632

In [120]:
# Turn precipitation data into one dataframe for whole of August

precip_august = pd.concat(precip_dataframes, ignore_index=True)
len(precip_august)

291648

In [128]:
# Add date thats in Y-M-D string format, and change column names to match fire dataframe

precip_august['acq_date'] = precip_august['time'].apply(str).apply(lambda x: x[:10])
precip_august = precip_august.rename(columns={'lon': 'longitude', 'lat':'latitude'})                                                         

In [129]:
# Import and prepare fire data

fires_2015 = pd.read_csv('./USA_fire_date_2010_2023/fire_data_2015.csv')
fires_2015['month'] = fires_2015['acq_date'].apply(lambda x: x[5:7])
fires_august_2015 = fires_2015[fires_2015['month'] == '08']

In [130]:

%%capture --no-stdout

# Round latitude and longitude values to nearest 0.05

fires_august_2015['longitude'] = fires_august_2015['longitude'].apply(lambda x: round(x/0.05) * 0.05).apply(lambda x: round(x,2))
fires_august_2015['latitude'] = fires_august_2015['latitude'].apply(lambda x: round(x/0.05) * 0.05).apply(lambda x: round(x,2))

In [131]:
# Since we are only looking at a day resolution on whether or not fire happened, drop duplicates

fires_august_2015 = fires_august_2015.drop_duplicates(subset = ['latitude','longitude'])
fires_august_2015

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,year,month
81942,642645,19.45,-155.10,335.9,3.1,1.7,2015-08-01,19.0,Aqua,MODIS,87.0,6.03,296.1,162.1,D,1.0,2015,08
81943,642646,19.40,-155.30,365.2,3.0,1.7,2015-08-01,19.0,Aqua,MODIS,100.0,6.03,306.9,459.7,D,1.0,2015,08
81944,642647,19.40,-155.05,317.5,3.2,1.7,2015-08-01,19.0,Aqua,MODIS,45.0,6.03,294.4,51.1,D,1.0,2015,08
81946,642649,19.40,-155.25,327.8,3.0,1.7,2015-08-01,19.0,Aqua,MODIS,31.0,6.03,300.1,97.4,D,1.0,2015,08
81949,642652,19.40,-155.10,318.8,3.1,1.7,2015-08-01,19.0,Aqua,MODIS,58.0,6.03,293.6,58.4,D,1.0,2015,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116791,677494,46.35,-105.15,321.4,1.5,1.2,2015-08-31,2025.0,Aqua,MODIS,65.0,6.03,303.7,19.6,D,0.0,2015,08
116794,677497,46.50,-105.20,334.3,1.5,1.2,2015-08-31,2025.0,Aqua,MODIS,84.0,6.03,302.2,47.7,D,0.0,2015,08
116797,677500,20.90,-156.50,310.7,3.8,1.8,2015-08-31,2029.0,Terra,MODIS,24.0,6.03,292.1,44.3,D,2.0,2015,08
116799,677502,20.60,-156.40,315.4,3.7,1.8,2015-08-31,2029.0,Terra,MODIS,0.0,6.03,294.2,46.8,D,0.0,2015,08


In [138]:
fires_california = fires_august_2015[fires_august_2015['latitude'] < 42][fires_august_2015['latitude']>33][fires_august_2015['longitude']<-115]
precip_august['fire'] = [0]*len(precip_august)

for index, row in fires_california.iterrows():
    match_condition = (precip_august['latitude'] == row['latitude']) & (precip_august['longitude'] == row['longitude']) & (precip_august['acq_date'] == row['acq_date'])
    precip_august.loc[match_condition, 'fire'] = 1
    
    print(index)
    clear_output(wait = True)
precip_august

,time,longitude,latitude,precipitationCal,acq_date,fire
0,2015-08-11 00:00:00,-124.25,32.25,0.0,2015-08-11,0
1,2015-08-11 00:00:00,-124.25,32.35,0.0,2015-08-11,0
2,2015-08-11 00:00:00,-124.25,32.45,0.0,2015-08-11,0
3,2015-08-11 00:00:00,-124.25,32.55,0.0,2015-08-11,0
4,2015-08-11 00:00:00,-124.25,32.65,0.0,2015-08-11,0
...,...,...,...,...,...,...
291643,2015-08-20 00:00:00,-114.75,41.55,0.0,2015-08-20,0
291644,2015-08-20 00:00:00,-114.75,41.65,0.0,2015-08-20,0
291645,2015-08-20 00:00:00,-114.75,41.75,0.0,2015-08-20,0
291646,2015-08-20 00:00:00,-114.75,41.85,0.0,2015-08-20,0


In [139]:
print(len(precip_august[precip_august['fire'] == 1]))
print(len(fires_california))

58
261


In [137]:
precip_august.to_csv('testie.csv')